سپهر سلمانی یگانه ۴۰۰۱۰۰۲۶۵

 سارا اکبری خرم ۴۰۰۱۰۹۳۰۵

In [297]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

دیتاها را لود می‌کنیم. با ترسنپوز کردن، هر سطر قیمت یک مارکت می‌شود

In [298]:
mat_data = sp.io.loadmat("All.mat")
all_price = np.array(mat_data['A']).transpose()
market_num = len(all_price)
data_num = len(all_price[0])
rtrn_num = data_num - 1

از روی قیمت برای هر مارکت ریترن را حساب می‌کنیم

In [299]:
def price_to_rtrn(price):
    rtrn = np.zeros(rtrn_num)
    for t in range(rtrn_num):
        rtrn[t] = price[t + 1] / price[t] - 1
    return rtrn

In [300]:
all_rtrn = np.zeros((market_num, rtrn_num))
for market_index in range(market_num):
    all_rtrn[market_index] = price_to_rtrn(all_price[market_index])

حال ریترن هر بازار را لوکال نرمالایز می‌کنیم. برای این کار ریترن هر بازار را در پنجره‌های ۱۳ روزه بررسی می‌کنیم. از آنجا که طول دیتا بر ۱۳ بخش‌پذیر نبود، پنجره‌ی آخر را تعداد روزهای باقی‌مانده گرفتیم

In [301]:
def rtrn_to_rLN(rtrn):
    windows, last_window = reshape_rtrn(rtrn)
    windows_num, window_len = windows.shape
    rLN = np.zeros(rtrn_num)
    for i in range(windows_num):
        rLN[i * window_len:(i + 1) * window_len] = local_normalize(windows[i])
    rLN[-len(last_window):] = local_normalize(last_window)
    return rLN

In [302]:
def local_normalize(window):
    window_len = len(window)
    mean = np.mean(window)
    std = np.std(window)
    LN = np.zeros(window_len)
    for i in range(window_len):
        LN[i] = (window[i] - mean) / std
    return LN

In [303]:
def reshape_rtrn(rtrn):
    window_len = 13
    last_window_len = rtrn_num % window_len
    last_window = rtrn[-last_window_len:]
    windows_num = rtrn_num // 13
    windows = rtrn[:-last_window_len].reshape(windows_num, window_len)
    return windows, last_window

In [304]:
all_rLN = np.zeros(np.shape(all_rtrn))
for market_index in range(market_num):
    all_rLN[market_index] = rtrn_to_rLN(all_rtrn[market_index])

حال کل دیتای تمام مارکت‌ها را به پنجره‌های ۴۴ روزه تقسیم می‌کنیم. بازهم پنجره آخر را تعداد روزهای باقی‌مانده گرفتیم

In [305]:
def reshape_rLN(rLN):
    rLN_transpose = np.transpose(rLN)
    window_len = 44
    windows_num = rLN_transpose.shape[0] // window_len
    windows = np.zeros((windows_num, rLN_transpose.shape[1], window_len))
    for i in range(windows_num):
        windows[i] = rLN_transpose[i * window_len:(i + 1) * window_len].transpose()
    last_window_len = rLN_transpose.shape[0] % window_len
    last_window = np.array(rLN_transpose[-last_window_len:].transpose())
    return windows, last_window

In [306]:
rLN_windows, rLN_last_window = reshape_rLN(all_rLN)

ماتریس‌های کراس کورلیشن را حساب می‌کنیم

In [307]:
correlation_matrices = np.zeros((len(rLN_windows) + 1, rLN_last_window.shape[0], rLN_last_window.shape[0]))
for window_index in range(len(correlation_matrices) - 1):
    correlation_matrices[window_index] = np.corrcoef(rLN_windows[window_index])
correlation_matrices[-1] = np.corrcoef(rLN_last_window)

هیتمپ این ماتریس‌ها را رسم و ذخیره می‌کنیم

In [308]:
for matrix_index in range(len(correlation_matrices)):
    sns.heatmap(correlation_matrices[matrix_index]).get_figure().savefig(f"heatmaps/heatmap-of-window{matrix_index}.png")
    plt.close()